# Avaliação

1. Crie uma função que recebe uma lista e retorna o valor máximo e mínimo da lista (não utilize a função max() e min()).
2. Crie uma função que recebe dois parâmetros (início e fim) e retorna uma lista com todos os valores pares entre ínicio e fim.
3. Acesse a nota de matemática em dc3 e altere para 3.

Para as questões 4 a 8 utilize o dataset 'water_potability.csv'. O dataset descreve características de uma amostra de água e na coluna 'potability' descreve se a água é ou não própria para consumo humano. Para iniciar a resolução das questões abaixo drope a coluna 'Unnamed: 0'.

4. Demonstre a correlação entre todas as colunas do dataset. Qual é a coluna com maior correlação positiva com 'potability'? Desconsidere a correlação com ela mesma (valor 1.0).
5. Crie uma série que descreve (em porcentagem) quanto cada coluna tem de valores nulos. Qual a coluna com maior porcentagem de valores nulos?
6. Crie uma coluna dentro do dataset que descreve se 'Turbidity' está acima ou abaixo da média.
7. Quantas amostras de água são potáveis e não potáveis, utilize a coluna criada na questão 6 como dimensão de análise.
8. Crie um dataframe onde o índice deve ser cada estado e com duas colunas. A primeira é a porcentagem de amostras não potáveis naquele estado e a segunda é a porcentagem de amostras potáveis naquele estado. Qual é o estado com a menor porcentagem de amostras de água potável?
9. Utlizando dados coletados da internet sobre a quantidade de habitantes de cada estado no ano 2020 responda. Qual estado, com populacao acima de 10 milhões de habitantes, tem a maior porcentagem de amostras de águas não potável?
10. Crie o dicionário dc_states contendo como chaves os nomes de cada estado e como valor uma lista contendo o zscore aplicado a coluna "Solids" de todas as amostras daquele respectivo estado. Passe para função violin_plot_states o dicionário e uma lista contendo 5 estados que gostaria de análisar. Faça uma análise sucinta do resultado do gráfico.

In [2]:
dc3 = { 
   "turma":{ 
      "aluno":{ 
         "nome":"Joao",
         "notas":{ 
            "portugues":2,
            "matematica":9
         }
      }
   }
}

import matplotlib.pyplot as plt
import seaborn as sns

def violin_plot_states(dc_states,lista_estados):
    f, ax = plt.subplots(1,1,figsize=(16,4))
    data_plot = pd.DataFrame.from_dict(dc_states,orient='index').T.melt().rename(columns={'variable':'state','value':'z_score'})
    states_plot = lista_estados
    sns.violinplot(x='state',y='z_score',data=data_plot[data_plot.state.isin(states_plot)])    
    ax.axhline(0,color='black',alpha=0.3)